In [ ]:
import pandas as pd
import torch
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM
from tqdm import tqdm

In [ ]:
model_id = "sivakornboonnet/ThaiT5-absa-thai-model"
tokenizer = AutoTokenizer.from_pretrained(model_id)
model = AutoModelForSeq2SeqLM.from_pretrained(model_id)

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/20.3k [00:00<?, ?B/s]

spiece.model:   0%|          | 0.00/784k [00:00<?, ?B/s]

added_tokens.json:   0%|          | 0.00/121 [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/1.30k [00:00<?, ?B/s]

/usr/local/lib/python3.11/dist-packages/transformers/convert_slow_tokenizer.py:559: UserWarning: The sentencepiece tokenizer that you are converting to a fast tokenizer uses the byte fallback option which is not implemented in the fast tokenizers. In practice this means that the fast version of the tokenizer can produce unknown tokens whereas the sentencepiece version would have converted these unknown tokens into a sequence of byte tokens matching the original piece of text.
  warnings.warn(


config.json:   0%|          | 0.00/833 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/977M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/142 [00:00<?, ?B/s]

In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = model.to(device)

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
df = pd.read_csv('/content/drive/MyDrive/KMITL/Project/Data/dataset_for_finetune_merged.csv')

In [ ]:
predictions = []

In [ ]:
for text in tqdm(df['text'], desc="🔄 Generating predictions"):
    inputs = tokenizer(text, return_tensors="pt", truncation=True, max_length=512).to(device)
    outputs = model.generate(**inputs, max_length=256)
    decoded = tokenizer.decode(outputs[0], skip_special_tokens=False).strip()

    # ลบ <pad> และ </s> ที่ไม่ต้องการ
    cleaned = decoded.replace("<pad>", "").replace("</s>", "").strip()

    predictions.append(cleaned)

🔄 Generating predictions: 100%|██████████| 4749/4749 [29:27<00:00,  2.69it/s]


In [ ]:
result_df = pd.DataFrame({
    'text': df['text'],
    'prediction': predictions,
    'label': df['label']  # Label จริง
})

In [ ]:
save_path = '/content/drive/MyDrive/KMITL/Project/Data/prediction_vs_label.csv'
result_df.to_csv(save_path, index=False, encoding='utf-8-sig')

print(f"✅ Prediction saved to: {save_path}")

✅ Prediction saved to: /content/drive/MyDrive/KMITL/Project/Data/prediction_vs_label.csv


In [ ]:
!pip install evaluate bert-score --quiet

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.0/84.0 kB 4.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.1/61.1 kB 3.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 487.4/487.4 kB 33.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 8.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 183.9/183.9 kB 13.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 143.5/143.5 kB 10.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 2.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 82.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 78.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 38.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 1.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 10.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [ ]:
import pandas as pd

df = pd.read_csv('/content/drive/MyDrive/KMITL/Project/Data/prediction_vs_label.csv')
preds = df['prediction'].astype(str).tolist()
labels = df['label'].astype(str).tolist()

In [ ]:
exact_match = [p.strip() == l.strip() for p, l in zip(preds, labels)]
exact_score = sum(exact_match) / len(exact_match)
print(f"✅ Exact Match Accuracy: {exact_score:.4f}")

✅ Exact Match Accuracy: 0.0000


In [ ]:
!pip install rouge_score --quiet # Install the rouge_score package to provide the necessary functionality for ROUGE score calculation.

  Preparing metadata (setup.py) ... done


In [ ]:
import evaluate
rouge = evaluate.load("rouge")
rouge_result = rouge.compute(predictions=preds, references=labels)
print("📊 ROUGE Score:")
for k, v in rouge_result.items():
    print(f"{k}: {v:.4f}")

📊 ROUGE Score:
rouge1: 0.8678
rouge2: 0.8375
rougeL: 0.8678
rougeLsum: 0.8678


In [ ]:
from bert_score import score
P, R, F1 = score(preds, labels, lang="th", verbose=True)
print(f"🧠 BERTScore (F1 mean): {F1.mean().item():.4f}")

tokenizer_config.json:   0%|          | 0.00/49.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/625 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/996k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.96M [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/714M [00:00<?, ?B/s]

calculating scores...
computing bert embedding.


  0%|          | 0/145 [00:00<?, ?it/s]

computing greedy matching.


  0%|          | 0/75 [00:00<?, ?it/s]

done in 15.67 seconds, 303.01 sentences/sec
🧠 BERTScore (F1 mean): 0.9318


In [ ]:
# เพิ่มคอลัมน์ exact_match เข้า DataFrame
df['exact_match'] = exact_match

# Export ไฟล์รวมผลประเมิน
save_path = '/content/drive/MyDrive/KMITL/Project/Data/prediction_vs_label_evaluated.csv'
df.to_csv(save_path, index=False, encoding='utf-8-sig')
print(f"📁 Evaluated file saved to: {save_path}")

📁 Evaluated file saved to: /content/drive/MyDrive/KMITL/Project/Data/prediction_vs_label_evaluated.csv
